In [ ]:
from azureml.core import Workspace
ws = Workspace(subscription_id="9d766322-e0aa-4e0d-9225-aa934bcc1b4a",
               resource_group="zhaw_AppliedDataScience",
               workspace_name="zhawadsMachineLearning")

In [ ]:
import urllib.request
from azureml.core.model import Model

# Download model
#urllib.request.urlretrieve("azureml-models/", "/vehicles_100.h5:1")
model1 = Model(ws, 'vehicles_100.h5')


model = Model.register(model_path=ws/'vehicles_100.h5',
                          model_name="vehicles_100.h5",
                          tags={'area': "vehicles", 'type': "cnn"},
                          description="vehicle rec.",
                          workspace=ws)


In [ ]:
# Download model
urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "model.onnx")

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

In [ ]:
model = run.register_model(model_name='bidaf_onnx',
                           tags={'area': 'qna'},
                           model_path='outputs/model.onnx')
print(model.name, model.id, model.version, sep='\t')

In [ ]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="project_environment")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./echo_score.py",
)

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

In [ ]:
print(service.get_logs())


## Call Model

In [57]:
import requests

uri = 'https://vehicles.eastus.inference.ml.azure.com/score'


headers = {"Content-Type": "application/json"}
data = {
    "query": "What color is the fox",
    "context": "The quick brown fox jumped over the lazy dog.",
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)